In [ ]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import glob
import time
import sys
import cobra

In [ ]:
# Generate 'robust' dataset
genome_ids = glob.glob('../metabolic_output/*.data')
genome_ids = [x.replace("../metabolic_output/","").replace(".data","") for x in genome_ids]

ID_list = []
# Write header to file
for prod_ind in list(range(0,35)):
    ID = str(str(prod_ind))
    ID_list.append(ID)

output_file = '../Data/stand_whole_path_avg_robust.tsv'  

output_file = open(output_file, 'w')
header = 'genomes\t' + '\t'.join(ID_list) + '\n'
output_file.write(header)

for genome_id in genome_ids[0:1]:
    file_name = "../metabolic_output/%s.data" % (genome_id)
    total_dataset_dict = pickle.load(open(file_name, "rb"))
    # Write row name
    output_file.write(genome_id + '\t')
    # Write whole_path_like to rows
    for prod_ind in list(range(0,35)):
        robust_output =[]
        for carb_ind in list(range(0,32)):
            ID = str(genome_id + ':' + str(carb_ind) + '.0.' + str(prod_ind))
            if ID != str(genome_id + ':16.0.25'):
                len(total_dataset_dict[ID]['gaps_w_flux'])
                len(total_dataset_dict[ID]['path_w_flux'])
                
                robust_output.append(round(total_dataset_dict[ID]['whole_path_like'],4))
        robust_avg = np.median(robust_output) # Median rather than mean
        if prod_ind != 34:
            output_file.write(str(round(robust_avg,4))+'\t')
        elif prod_ind == 34:
            output_file.write(str(round(robust_avg,4))+'\n')
        else:
            print('ERROR')

output_file.close()